In [2]:
import tensorflow as tf
import numpy as np
import nibabel as nib
import os
import regex as re
from scipy.ndimage import zoom
from tensorflow.keras import layers, Model, Input
import pickle
from tqdm import tqdm

In [4]:
# string with pattern (starting with numbers)
re_number = re.compile(r'^\d+')
# string with pattern (starting with numbers, (can be anything in middle) ending with nii)
re_swar_number_nii = re.compile(r"^swar\d+.+\.nii$")

In [17]:
for num, i in enumerate(func_features):
    if (i != np.array([ 79,  95,  79, 197])) .any():
        print(i, num)

[ 47  56  47 197] 26


In [ ]:
struct_features = []
struct_shapes = []
cnt = 0
main_dirs = [r"./MCI (1year)", r"./MCI (2years)", r"./MCI (3years)"]
for main_dir in main_dirs:
    for sub in os.listdir(main_dir):
        if not os.path.isdir(os.path.join(main_dir, sub)) or not re_number.match(sub):
            continue
        if sub.find("--") != -1:
            continue
        subject_path = os.path.join(main_dir,sub)
        for struct_func in os.listdir(subject_path):
            if not struct_func.startswith("struct"):
                continue
            func_dir = os.path.join(subject_path, struct_func)
            cnt += 1
            for mri_file in os.listdir(func_dir):
                if not re_number_nii.match(mri_file):
                    continue
                lovda_img = os.path.join(func_dir,mri_file)
                func_image = nib.load(lovda_img)
                struct_shapes.append(func_image.shape)
                struct_features.append(func_image.get_fdata())

In [6]:
struct_features[0].shape

(208, 240, 256)

In [7]:
desired_shape = (150, 150, 150)

# Compress all images and add channel dimension
compressed_features = []
for image in struct_features:
    zoom_factors = [desired_shape[i] / image.shape[i] for i in range(3)]
    compressed_image = zoom(image, zoom_factors, order=3)
    compressed_image = np.expand_dims(compressed_image, axis=-1)  # Add channel dimension
    compressed_features.append(compressed_image)

# Convert to a NumPy array
compressed_features_array = np.array(compressed_features)

In [8]:
compressed_features_array.shape

(53, 150, 150, 150, 1)

In [26]:
def model_3d_fc(input_shape_3d=(150, 150, 150, 1), input_shape_2d=(116, 116, 1), num_classes=3):
    # 3D Structural Image Input
    input_3d = Input(shape=input_shape_3d, name='3d_image_input')
    x = layers.Conv3D(32, (3, 3, 3), activation='relu')(input_3d)
    x = layers.MaxPooling3D((2, 2, 2))(x)
    x = layers.Conv3D(64, (3, 3, 3), activation='relu')(x)
    x = layers.MaxPooling3D((2, 2, 2))(x)
    x = layers.Conv3D(128, (3, 3, 3), activation='relu')(x)
    x = layers.MaxPooling3D((2, 2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    
    # 2D Functional Connectivity Map Input
    input_2d = Input(shape=input_shape_2d, name='2d_fc_input')
    y = layers.Conv2D(32, (3, 3), activation='relu')(input_2d)
    y = layers.MaxPooling2D((2, 2))(y)
    y = layers.Conv2D(64, (3, 3), activation='relu')(y)
    y = layers.MaxPooling2D((2, 2))(y)
    y = layers.Conv2D(128, (3, 3), activation='relu')(y)
    y = layers.MaxPooling2D((2, 2))(y)
    y = layers.Flatten()(y)
    y = layers.Dense(128, activation='relu')(y)
    
    # Concatenate the outputs of both branches
    concatenated = layers.concatenate([x, y])
    
    # Final classification layer
    z = layers.Dense(128, activation='relu')(concatenated)
    z = layers.Dropout(0.5)(z)
    output = layers.Dense(num_classes, activation='softmax')(z)
    
    # Create the model
    model = Model(inputs=[input_3d, input_2d], outputs=output)
    
    return model

# Define the model
input_shape_3d = (150, 150, 150, 1)  # Shape of the 3D structural image
input_shape_2d = (116, 116, 1)  # Shape of the 2D functional connectivity map
num_classes = 3  # Number of categories for classification

model = model_3d_fc(input_shape_3d=input_shape_3d, input_shape_2d=input_shape_2d, num_classes=num_classes)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()




Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 3d_image_input (InputLayer  [(None, 150, 150, 150, 1)]   0         []                            
 )                                                                                                
                                                                                                  
 2d_fc_input (InputLayer)    [(None, 116, 116, 1)]        0         []                            
                                                                                                  
 conv3d (Conv3D)             (None, 148, 148, 148, 32)    896       ['3d_image_input[0][0]']      
                                                                                                  
 conv2d (Conv2D)             (None, 114, 114, 32)         320       ['2d_fc_input[0][0]']  

In [9]:
mci_func = pickle.load(open(r"./feature_extraction\MCI_final_features_all_1.pkl", "rb"))
ad_func = np.array(pickle.load(open(r"./feature_extraction\func_features_AD.pkl", "rb")))
mci_func.shape, ad_func.shape

((53, 13456), (40, 13456))

In [51]:
type(ad_func)

numpy.ndarray

In [52]:
pickle.dump(ad_func, open(r"./feature_extraction\func_features_AD.pkl", "wb"))

In [10]:
func_features = np.concatenate((mci_func, ad_func), axis=0)
func_features = func_features.reshape(func_features.shape[0], 116, 116, 1)
func_features.shape

(93, 116, 116, 1)

In [11]:
struct_features_ad = pickle.load(open(r"./feature_extraction\Compressed_features.pkl", "rb"))

In [12]:
struct_features_ad.shape

(40, 150, 150, 150, 1)

In [13]:
struct_features = np.concatenate((compressed_features_array, struct_features_ad), axis=0)
struct_features.shape

(93, 150, 150, 150, 1)

In [14]:
# train test split
from sklearn.model_selection import train_test_split

In [15]:
y = np.concatenate((np.zeros(mci_func.shape[0]), np.ones(ad_func.shape[0])), axis=0)
y.shape

(93,)

In [16]:
y[y==1].shape, y[y==0].shape

((40,), (53,))

In [17]:
X = []
for i in range(struct_features.shape[0]):
    X.append([struct_features[i], func_features[i]])

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
len(X_train)

74

In [20]:
real_x_train_struct = [X_train[i][0] for i in range(len(X_train))]
real_x_train_func = [X_train[i][1] for i in range(len(X_train))]

In [21]:
real_x_train_struct = np.array(real_x_train_struct)
real_x_train_func = np.array(real_x_train_func)
real_x_train_struct.shape, real_x_train_func.shape

((74, 150, 150, 150, 1), (74, 116, 116, 1))

In [58]:
history = model.fit([real_x_train_struct, real_x_train_func], y_train, epochs=20, batch_size=8)

Epoch 1/20
10/10 [==============================] - 123s 10s/step - loss: 0.1695 - accuracy: 0.9459
Epoch 2/20
10/10 [==============================] - 84s 8s/step - loss: 0.0223 - accuracy: 0.9865
Epoch 3/20
10/10 [==============================] - 85s 8s/step - loss: 0.0362 - accuracy: 0.9865
Epoch 4/20
10/10 [==============================] - 83s 8s/step - loss: 0.0143 - accuracy: 1.0000
Epoch 5/20
10/10 [==============================] - 84s 8s/step - loss: 0.0116 - accuracy: 1.0000
Epoch 6/20
10/10 [==============================] - 89s 9s/step - loss: 0.0223 - accuracy: 0.9865
Epoch 7/20
10/10 [==============================] - 86s 9s/step - loss: 0.0779 - accuracy: 0.9595
Epoch 8/20
10/10 [==============================] - 84s 8s/step - loss: 0.0781 - accuracy: 0.9595
Epoch 9/20
 1/10 [==>...........................] - ETA: 1:34 - loss: 0.0035 - accuracy: 1.0000

KeyboardInterrupt: 

In [22]:
# testing the model
real_x_test_struct = [X_test[i][0] for i in range(len(X_test))]
real_x_test_func = [X_test[i][1] for i in range(len(X_test))]
real_x_test_struct = np.array(real_x_test_struct)
real_x_test_func = np.array(real_x_test_func)

In [23]:
# testing the model
model.evaluate([real_x_test_struct, real_x_test_func], y_test)


1/1 [==============================] - 89s 89s/step - loss: 1.3218 - accuracy: 0.5789


[1.321844458580017, 0.5789473652839661]

In [24]:
model.evaluate([real_x_train_struct, real_x_train_func], y_train)

3/3 [==============================] - 429s 63s/step - loss: 0.0281 - accuracy: 1.0000


[0.028146076947450638, 1.0]

In [60]:
model.save("./feature_extraction/3d_fc_model.h5")

c:\Users\maddi\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
model = tf.keras.models.load_model("./feature_extraction/3d_fc_model.h5")

# 3d MRI + 1d connectivity map

In [25]:
def model_3d_fc_1d(input_shape_3d=(150, 150, 150, 1), input_shape_1d=(116 * 116), num_classes=3):
    # 3D Structural Image Input
    input_3d = Input(shape=input_shape_3d, name='3d_image_input')
    x = layers.Conv3D(32, (3, 3, 3), activation='relu')(input_3d)
    x = layers.MaxPooling3D((2, 2, 2))(x)
    x = layers.Conv3D(64, (3, 3, 3), activation='relu')(x)
    x = layers.MaxPooling3D((2, 2, 2))(x)
    x = layers.Conv3D(128, (3, 3, 3), activation='relu')(x)
    x = layers.MaxPooling3D((2, 2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    
    # 1d functional connectivity array
    input_1d = Input(shape=input_shape_1d, name='1d_fc_input')
    y = layers.Dense(256, activation='relu')(input_1d)
    y = layers.Dropout(0.5)(y)
    y = layers.Dense(128, activation='relu')(y)
    
    # Concatenate the outputs of both branches
    concatenated = layers.concatenate([x, y])
    
    # Final classification layer
    z = layers.Dense(128, activation='relu')(concatenated)
    z = layers.Dropout(0.5)(z)
    output = layers.Dense(num_classes, activation='softmax')(z)
    
    # Create the model
    model = Model(inputs=[input_3d, input_1d], outputs=output)
    
    return model

# Define the model
input_shape_3d = (150, 150, 150, 1)  # Shape of the 3D structural image
input_shape_1d = (116 * 116)  # Shape of the 2D functional connectivity map
num_classes = 3  # Number of categories for classification

model = model_3d_fc_1d(input_shape_3d=input_shape_3d, input_shape_1d=input_shape_1d, num_classes=num_classes)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 3d_image_input (InputLayer  [(None, 150, 150, 150, 1)]   0         []                            
 )                                                                                                
                                                                                                  
 conv3d (Conv3D)             (None, 148, 148, 148, 32)    896       ['3d_image_input[0][0]']      
                                                                                                  
 max_pooling3d (MaxPooling3  (None, 74, 74, 74, 32)       0         ['conv3d[0][0]']              
 D)                                                                                               
                                                                                             

In [38]:
func_features = func_features.reshape(func_features.shape[0], 116 * 116)

In [40]:
X = []
for i in range(struct_features.shape[0]):
    X.append([struct_features[i], func_features[i]])

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
real_x_train_struct_2 = [X_train[i][0] for i in range(len(X_train))]
real_x_train_func_2 = [X_train[i][1] for i in range(len(X_train))]
real_x_train_struct_2 = np.array(real_x_train_struct_2)
real_x_train_func_2 = np.array(real_x_train_func_2)

real_x_test_struct = [X_test[i][0] for i in range(len(X_test))]
real_x_test_func = [X_test[i][1] for i in range(len(X_test))]
real_x_test_struct_2 = np.array(real_x_test_struct)
real_x_test_func_2 = np.array(real_x_test_func)

In [47]:
history2 = model.fit([real_x_train_struct_2, real_x_train_func_2], y_train, epochs=10, batch_size=8)

Epoch 1/10

10/10 [==============================] - 288s 13s/step - loss: 864.3452 - accuracy: 0.5000
Epoch 2/10
10/10 [==============================] - 100s 10s/step - loss: 1.0877 - accuracy: 0.5541
Epoch 3/10
10/10 [==============================] - 88s 9s/step - loss: 1.0096 - accuracy: 0.4595
Epoch 4/10
10/10 [==============================] - 88s 9s/step - loss: 0.7914 - accuracy: 0.5946
Epoch 5/10
10/10 [==============================] - 85s 8s/step - loss: 0.8480 - accuracy: 0.5270
Epoch 6/10
10/10 [==============================] - 83s 8s/step - loss: 0.8424 - accuracy: 0.6216
Epoch 7/10
10/10 [==============================] - 83s 8s/step - loss: 0.7861 - accuracy: 0.6216
Epoch 8/10
10/10 [==============================] - 80s 8s/step - loss: 0.7085 - accuracy: 0.6351
Epoch 9/10
10/10 [==============================] - 87s 9s/step - loss: 0.7641 - accuracy: 0.6622
Epoch 10/10
10/10 [==============================] - 96s 10s/step - loss: 0.7678 - accuracy: 0.7027


In [48]:
# test accuracy
model.evaluate([real_x_test_struct_2, real_x_test_func_2], y_test)

1/1 [==============================] - 56s 56s/step - loss: 0.7856 - accuracy: 0.6842


[0.7855609655380249, 0.6842105388641357]

In [49]:
# train accuracy
model.evaluate([real_x_train_struct_2, real_x_train_func_2], y_train)

3/3 [==============================] - 514s 118s/step - loss: 0.6242 - accuracy: 0.6757


[0.6242111921310425, 0.6756756901741028]

In [ ]:
# given two arrays of numbers firstarray and secondarray, return the length of the longest common prefix (LCP) between any pair of numbers from different arrays or 0 if no common prefix exists.

